## Introduction
The recent development of Hi-C and related methods have revealed substantial hierarchical structures of 3D chromatin organization, such as compartments, sub-compartments, topologically associating domains, and loops. However, we still know little about the heterogeneity and variability of these structures in single cells as the measurements derived from Hi-C are based on the population of millions of cells. For example, to what extent the Hi-C interaction frequency related to the physical distance for two genomic loci. To answer these kinds of questions,  Finn et al. applied HiFISH, a high-throughput imaging method, to assess the relationship between the spatial location and colocalization frequencies for more than a hundred pairs of genomic loci in single-cell resolution. 

In this notebook, we aim to use the Nucleome Browser to interactively explore the processed data from Finn et al. Specifically, we will use the Python notebook to examine the correlation between Hi-C interaction frequency and contact frequency of pairs of genomic loci. Nucleome Browser has a specific panel hosting microscopy data from the DCIC data portal. We will use this panel to explore raw FISH images on Nucleome Browser and view them interactively inside the notebook. We will also use a customized web page to view the Hi-C matrix on the HiGlass platform. Notebook, FISH images on Nucleome Browser, and Hi-C contact matrix in HiGlass are all connected through the unique 'event-driven communication' developed in the Nucleome Browser.

## Microscopy Data Preprocessing
In this section, we will retrieve the Hi-C data and processed image data of Finn et al. We will use the 4DN DCIC Jupyter notebook to directly access the data. THe script in this sections requires access to the 4DN DCIC Jupyter Hub. You also prepare the processed data if you cannot run this notebook in the 4DN DCIC Jupyter notebook server. You can skip this session and load the processed data directly.

In [1]:
from dcicutils import jh_utils
import numpy
import pandas as pd
import cooler

# the mcool file from the dataset
file_path_hic = jh_utils.mount_4dn_file('4DNFIMDOXUT8')

# the summary results file for microscopy images
file_path_mic = jh_utils.mount_4dn_file('4DNFID78GRES')

We then load the processed file for the microscopy data.

In [2]:
# Load the processed data
mic_data = pd.read_csv(file_path_mic, sep = '\t', header = 0)
mic_data.head(3)

,Bait ID,Bait Chr,Bait Start,Bait End,Target ID,Target Chr,Target Start,Target End,Dataset,Hi-C Capture Frequency,...,Fig 1 (E-I),Fig 1 (J-L),Fig 2,Fig 3,Fig 4,Fig 5,Fig 6,Bait,Target,chr
0,misteli:probe_11,1,2370445,2570720,misteli:probe_74,1,18566409,18766880,Chr1GR,0.001959,...,NaN,All,"D,E",All,NaN,NaN,NaN,11.0,74.0,1
1,misteli:probe_11,1,2370445,2570720,misteli:probe_91,1,22223362,22394657,Chr1GR,0.000818,...,NaN,All,"D,E",All,NaN,NaN,NaN,11.0,91.0,1
2,misteli:probe_11,1,2370445,2570720,misteli:probe_441,1,109469002,109662490,Chr1GR,0.000194,...,NaN,All,"D,E",All,NaN,NaN,NaN,11.0,441.0,1


In [3]:
# check column names
mic_data.columns

Index(['Bait ID', 'Bait Chr', 'Bait Start', 'Bait End', 'Target ID',
       'Target Chr', 'Target Start', 'Target End', 'Dataset',
       'Hi-C Capture Frequency', 'O/E Hi-C', 'Category', 'Gene Content',
       'Bias Quantile', 'Genomic Distance', 'Bait Compartment',
       'Target Compartment', 'nspots', '% within 150 nm (3D)',
       '% within 200 nm (3D)', '% within 350 nm (3D)', '% within 1 µm (3D)',
       'Mean 3D distance', 'Fig 1 (E-I)', 'Fig 1 (J-L)', 'Fig 2', 'Fig 3',
       'Fig 4', 'Fig 5', 'Fig 6', 'Bait', 'Target', 'chr'],
      dtype='object')

In [4]:
# check how many pairs of loci
mic_data.shape

(128, 33)

There are in total 128 pairs of loci. We then add unique IDs (strings) for each bait and target by concatenating chromosome name, start, and ending position into the format 'chr:start-end' following the style of the UCSC Genome Browser.

In [5]:
mic_data['Bait_string'] = 'chr' + mic_data['Bait Chr'].astype(str) + ':' + mic_data['Bait Start'].astype(str) + '-' + mic_data['Bait End'].astype(str)
mic_data['Target_string'] = 'chr' + mic_data['Target Chr'].astype(str) + ':' + mic_data['Target Start'].astype(str) + '-' + mic_data['Target End'].astype(str)

Check the string of Bait and Target.

In [6]:
mic_data.loc[0:5, ['Bait ID', 'Bait_string', 'Target ID', 'Target_string']]

,Bait ID,Bait_string,Target ID,Target_string
0,misteli:probe_11,chr1:2370445-2570720,misteli:probe_74,chr1:18566409-18766880
1,misteli:probe_11,chr1:2370445-2570720,misteli:probe_91,chr1:22223362-22394657
2,misteli:probe_11,chr1:2370445-2570720,misteli:probe_441,chr1:109469002-109662490
3,misteli:probe_11,chr1:2370445-2570720,misteli:probe_994,chr1:248125293-248263553
4,misteli:probe_52,chr1:12708719-12865743,misteli:probe_11,chr1:2370445-2570720
5,misteli:probe_52,chr1:12708719-12865743,misteli:probe_74,chr1:18566409-18766880


## Calculate Hi-C interaction intensity
In this section, we will get the corresponding Hi-C interaction intensity for these 128 pairs of loci.

For each bait and target, we first identify the closest bin ID that covers the region between the bait and the target. We then retrieve the distance normalized contact frequencies. 

In [7]:
def get_hic_intensity(mcoolfile, mic_data, resolution, valuetype):
    hic = cooler.Cooler(mcoolfile + '::resolutions/' + str(resolution))
    result = []
    for index, row in mic_data.iterrows():
        start_1 = list(hic.bins().fetch(row['Bait_string']).index)[0]
        end_1 = list(hic.bins().fetch(row['Bait_string']).index)[-1] + 1
        start_2 = list(hic.bins().fetch(row['Target_string']).index)[0]
        end_2 = list(hic.bins().fetch(row['Target_string']).index)[-1] + 1
        # normalized frequencies
        values = hic.matrix(balance=True, as_pixels=True, join=True)[start_1:end_1, start_2:end_2][valuetype]
        # average per-bin
        nbins1 = end_1 - start_1
        nbins2 = end_2 - start_2
        s = numpy.nansum(values)   # sum of per-bin intensities
        m = s / (nbins1 * nbins2)   # per-bin intensity
        m = m / resolution   # control for resolution effect
        result.append(m)
    return result
# get the normalized Hi-C interaction intensity
hic_intensity = get_hic_intensity(file_path_hic, mic_data, 10000, valuetype = 'balanced')
mic_data['hic_10k'] = hic_intensity

In [9]:
data = mic_data[['Bait_string', 'Target_string', 'hic_10k', '% within 350 nm (3D)']]
data.columns = ['Bait_string', 'Target_string', 'hic_interaction', 'fraction_ls_350']
# save the processed data into a file
data.to_csv("Finn_et_all_processed_data.tsv", sep = '\t', header = True, index = False)
# convert 
data.head()

,Bait_string,Target_string,hic_interaction,fraction_ls_350
0,chr1:2370445-2570720,chr1:18566409-18766880,3.692195e-09,2.4120
1,chr1:2370445-2570720,chr1:22223362-22394657,1.133845e-09,0.8362
2,chr1:2370445-2570720,chr1:109469002-109662490,5.075978e-10,1.0030
3,chr1:2370445-2570720,chr1:248125293-248263553,5.806794e-10,1.4553
4,chr1:12708719-12865743,chr1:2370445-2570720,0.000000e+00,5.0270


## Create an interactive scatterplot by linking to the Nucleome Browser
In this section, we will plot an interactive scatterplot with X-axis as the fraction of genomic loci within 350nm in spatial distance and Y-axis as the calculated Hi-C interaction intensity. Users can then click the points and view the Nucleome Browser will navigate to the corresponding regions and highlight the bait and target probe.

You can directly load the processed data if you do not have the access to the 4DN DCIC Jupyter server.

> **Nucleome Browser (version later than v0.9.9) supports the users to set channel ID for communication. In this demo, we will use the default communication channel (N-cnbChan01). If you want to use other channel (e.g., N-cnbChan02). you need to switch the channel using dispatch.chanId("<new channel>").**

In [2]:
import pandas as pd
data = pd.read_csv("data/Finn_et_all_processed_data.tsv", sep = '\t', header = 0)

In [3]:
from IPython.display import display, Javascript, HTML
import json

In [4]:
%%javascript
require.config({
    paths: {
        'd3': ['https://d3js.org/d3.v5.min'],
        'nb': ['https://vis.nucleome.org/static/lib/nb-dispatch'],
        'Plotly': ['https://cdn.plot.ly/plotly-latest.min']
    }
});

<IPython.core.display.Javascript object>

In [5]:
Javascript("""
(function(element){
    require(['d3', 'nb', 'Plotly'], function(d3, nb, Plotly) {
        // init div for plot
        d3.select(element[0]).append('div')
            .attr('id', 'myDiv')
        var myPlot = document.getElementById('myDiv');
        // prepare data
        var df = %s;
        var bait_string = d3.map(df['Bait_string']).entries().map(x=>x.value),
            target_string = d3.map(df['Target_string']).entries().map(x=>x.value),
            dataAxisX = d3.map(df['fraction_ls_350']).entries().map(x=>x.value/100),
            dataAxisY = d3.map(df['hic_interaction']).entries().map(x=>x.value);
        // get color for data in different chromosomes
        var chromID = [];
        var region = [];
        for (var i = 0; i < bait_string.length;  i++) {
            region = bait_string[i].split(/[-:]/);
            chromID.push(region[0]);
        }
        // create custom data
        var annoData = []
        for (var i = 0; i < bait_string.length; i++) {
            annoData.push([bait_string[i], target_string[i]]);
        }
        // prepare nb
        var out = d3.select(element.get(0)).append('div')
            .attr("id", "out")
        var nbHub = nb.dispatch("update","brush")
        nbHub.chanId('cnbChan01')
        nbHub.connect(function(d){
          d3.select("#out").html = d.connection
        })
        // plot data
        var data = [
            {
                type: 'scatter',
                mode: 'markers',
                x: dataAxisX,
                y: dataAxisY,
                customdata: annoData,
                hovertemplate: '<i>X</i>: %%{x:.2f}' +
                               '<br><i>Y</i>: %%{y:.4e}' +
                               '<br><i>Bait</i>: %%{customdata[0]}' +
                               '<br><i>Target</i>: %%{customdata[1]}',
                marker: {
                    size:9,
                    opacity: 0.7,
                    line: {
                        color: 'rgba(0,0,0,0.5)',
                        width: 2
                    }
                },
                transforms: [{type: "groupby", groups: chromID}],
                showlegend: true
            }
        ]
        var layout = {
            hovermode:'closest',
            title: "Comparison of Hi-C and Microscopy datasets",
            xaxis: {
                tickformat: '%%',
                title: "Percentage of distance under 350nm",
            },
            yaxis: {
                type: 'log',
                tickformat: '.1e',
                autorange: true
            },
            legend: {orientation: 'h', y: -0.3}
        };
        // plot function
        Plotly.newPlot('myDiv', data, layout);
        // handle click event
        myPlot.on('plotly_click', function(data) {
            var bait = data.points[0].customdata[0];
            var target = data.points[0].customdata[1];
            var regionBait = bait.split(/[-:]/);
            var regionTarget = target.split(/[-:]/);
            // first navigate to that region
            var navStart = Math.min(regionBait[1], regionTarget[1]) - 1000000;
            var navEnd = Math.max(regionBait[2], regionTarget[2]) + 1000000;
            nbHub.call("update", this, [{
                chr: regionBait[0], 
                start: navStart, 
                end: navEnd}]);
            nbHub.call("brush", this, [
                {
                chr: regionBait[0],
                start: regionBait[1],
                end: regionBait[2]
                },
                {
                chr: regionTarget[0],
                start: regionTarget[1],
                end: regionTarget[2]
                }])
        })
    });
})(element);
""" % (data.to_json(orient = 'columns')))

<IPython.core.display.Javascript object>

## Results

The scatterplot in the notebook demonstrates the positive relationship between the Hi-C contact frequency (Y-axis) and the fraction of pair of genomic loci within 350nm (X-axis). Those pairs with high Hi-C contact frequency are also more likely to be close to each other. However, we also noticed that the fraction on the X-axis is low indicating high variability in single cells. 

We then select a pair on chromosome 1 and click it. The Nucloeme Browser (right) and the HiGlass panel in the middle automatically navigate to that regions and two loci with probes are highlighted in these two panels. In the Nucleome Browser, the bottom right is a specific panel hosting the raw images generated in the Finn et al. paper. Each track is one experiment with a different combination of probes. We select an image from the second track in which two probes are tested (red bars above the image track). The OMERO.iviwer window will pop out and we can further explore this raw FISH image.

![Animation](img/NB_Jupyter_Finn_et_al_opt_1.8x.gif "Ainimation 1")